In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
DataCoSupplyChainDataset = pd.read_csv("../input/dataco-smart-supply-chain-for-big-data-analysis/DataCoSupplyChainDataset.csv", header= 0,encoding= 'unicode_escape')

In [ ]:
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score,r2_score,mean_absolute_error,mean_squared_error,accuracy_score,classification_report,confusion_matrix

In [ ]:
DataCoSupplyChainDataset.describe()


In [ ]:
DataCoSupplyChainDataset.shape

In [ ]:
DataCoSupplyChainDataset.head()

In [ ]:
DataCoSupplyChainDataset.info

In [ ]:
DataCoSupplyChainDataset.isnull().any()

In [ ]:
def data_info(data):
    print('Number of columns : ',data.shape[1])
    print('Number of rows  : ',data.shape[0])
    print('Total number of data-points :',data.size)
    numerical_features = [f for f in data.columns if data[f].dtypes!='O']
    print('Count of Numerical Features :',len(numerical_features))
    cat_features = [c for c in data.columns if data[c].dtypes=='O']
    print('Count of Categorical Features :',len(cat_features))
data_info(DataCoSupplyChainDataset)


In [ ]:
def features_with_missing_values(data):
    x = data.isnull().sum().sum()/(data.shape[0]*data.shape[1])*100
    print('Percentage of Total Missing Values ' ,round(x,2) ,'%')
    print('Missing Value Information :')
    for i in data.columns:
        if data[i].isna().sum()>0:
            print(i + " = " +str(data[i].isna().sum()))
            
features_with_missing_values(DataCoSupplyChainDataset)

In [ ]:
DataCoSupplyChainDataset['order_date'] = pd.to_datetime(DataCoSupplyChainDataset['order date (DateOrders)'])
DataCoSupplyChainDataset['shipping_date']=pd.to_datetime(DataCoSupplyChainDataset['shipping date (DateOrders)'])

In [ ]:

DataCoSupplyChainDataset['order_year'] = pd.DatetimeIndex(DataCoSupplyChainDataset['order_date']).year
DataCoSupplyChainDataset['order_month'] = pd.DatetimeIndex(DataCoSupplyChainDataset['order_date']).month
DataCoSupplyChainDataset['order_day'] = pd.DatetimeIndex(DataCoSupplyChainDataset['order_date']).day
DataCoSupplyChainDataset['shipping_year'] = pd.DatetimeIndex(DataCoSupplyChainDataset['shipping_date']).year
DataCoSupplyChainDataset['shipping_month'] = pd.DatetimeIndex(DataCoSupplyChainDataset['shipping_date']).month
DataCoSupplyChainDataset['shipping_day'] = pd.DatetimeIndex(DataCoSupplyChainDataset['shipping_date']).day

In [ ]:
new_dataset_features = ['Type','Days for shipping (real)','Days for shipment (scheduled)','Late_delivery_risk','Benefit per order',
                        'Sales per customer','Latitude','Longitude','Shipping Mode','Order Status','Order Region',
                        'Order Country','Order City','Market','Delivery Status','order_day','order_month','order_year',
                        'shipping_day','shipping_month','shipping_year']
len(new_dataset_features)

In [ ]:
DataCoSupplyChainDataset = DataCoSupplyChainDataset[new_dataset_features]
DataCoSupplyChainDataset.head()

In [ ]:
model_data = pd.get_dummies(DataCoSupplyChainDataset)
model_data.shape

In [ ]:
x = model_data.drop(['Days for shipping (real)','Days for shipment (scheduled)'],axis=1)
y = model_data[['Days for shipping (real)','Days for shipment (scheduled)']]
x.shape,y.shape

In [ ]:
x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,test_size=0.15)

In [ ]:
model=DecisionTreeRegressor()
model.fit(x_train,y_train)
pred=model.predict(x_test)

In [ ]:
#Converting the predicted output array to dataframe
Prediction = pd.DataFrame(pred)
prediction = Prediction.rename(columns={0:'Fastest_shipment',1:'Avg_shipment'})
prediction.head()

In [ ]:
prediction['risk'] = np.where(prediction['Avg_shipment'] >= prediction['Fastest_shipment'],0,1)

In [ ]:
prediction.head()

In [ ]:
print("Fraction of Delivery that are expected to deliver late  = ",+len(prediction[prediction['risk']==1])/len(prediction))

In [ ]:
l = prediction['risk']
m = x_test['Late_delivery_risk']
l.shape,m.shape

In [ ]:
def evaluation_risk_factor(l,m):
  print('The accuracy of model is :',accuracy_score(l,m))
  print(classification_report(l,m))
  ax=plt.subplot()
  sns.heatmap(confusion_matrix(l,m),annot=True,ax=ax);
  ax.set_xlabel('Predicted labels'); ax.set_ylabel('True labels')
  ax.set_title('Confusion matrix');

In [ ]:
evaluation_risk_factor(l,m)